Импортируем модули и глобальные переменные на уровне модулей.


In [ ]:
from src import data_utils, data_set, lstm_model, transformer
import requests

TOKENIZER = data_set.TOKENIZER
MIN_LEN = data_set.MIN_LEN
ROUGE = lstm_model.ROUGE

Готовим данные:
- Загружаем датасет.
- Чистим его с помощью регулярных выражений.
- Разбиваем на сеты для обучения и валидации.

In [ ]:
url = "https://code.s3.yandex.net/deep-learning/tweets.txt"
filename = "data/raw_dataset.txt"
response = requests.get(url, stream=True)
response.raise_for_status()

In [ ]:
# Аргумент cap ограничивает число строк, которые будут загружены.
# Значения до 100К удобны для проверки. Тетрадка отработает за несколько минут.
clean = data_utils.clean_up(data_utils.load_dataset(
    "data/raw_dataset.txt", cap=10000))

splits = data_utils.split_dataset(clean)

data_sets = []
data_loaders = []
for split in [splits['train'], splits['val']]:
    dset, dloader = data_set.prepare_data(split)
    data_sets.append(dset)
    data_loaders.append(dloader)

Обучаем LSTM модель

In [ ]:
lstm = lstm_model.Lstm(TOKENIZER)
lstm_model.train(lstm, n_epochs=3, l_rate=0.002, tokenizer=TOKENIZER,
                 train_loader=data_loaders[0], val_loader=data_loaders[1])

В ходе обучения предсказываем 3 слова для каждого предложения.

Почему так ? В задании сказано "предсказать оставшуюся 1/4 часть предложения". \
Т. к. предложения разной длины, то элементы батча будут обработаны разное число раз. \
Это ограничит возможности использования батчей, вот возможные варианты:
- Отказаться от батчей вовсе.
- Отсортировать предложения по длине и использовать маленький батч (чтобы длины в батче совпадали)
- Зафиксирваоть число слов (1/4 минимальной длины предложения или максмальной и т. п.)


В общем, получается логика обработки, которая не имеет никакого отношения к обучению. \
Если вы настаиваете, то я решу задачу одним из описанных способов.

In [ ]:
NUM_WORDS = 3

Инференс с авторегрессией

In [ ]:
# Заполняем датасет для инференса, т. к. мы обучаем модель предсказывать только
# 1 слово, а инференс хотим делать для нескольких.
lstm_split = []
true_split = []
for i in range(0, len(splits['test'])):
    words = splits['test'][i].split()
    if len(words) < MIN_LEN + NUM_WORDS:
        continue
    ctx_len = len(words) - NUM_WORDS
    context = ' '.join(words[:ctx_len])
    target = ' '.join(words[ctx_len:])
    lstm_split.append(context)
    true_split.append(context + ' ' + target)

# Данные для инференса трансформера будут точно такие же, делаем копию.
gpt2_split = lstm_split

# Проходимся по всему датасету и генерируем массив предсказанных слов.
# Далее, прибавляем предсказанное слово ко входам.
for i in range(0, NUM_WORDS):
    _, dloader = data_set.prepare_data(
        lstm_split, shuffle=False, num_targets=0)
    preds = lstm_model.inference(lstm, loader=dloader, tokenizer=TOKENIZER)
    lstm_split = [x + ' ' + y for x, y in zip(lstm_split, preds)]

rouge_score = ROUGE.compute(predictions=lstm_split, references=true_split)
print('ROUGE metrics')
for k, v in rouge_score.items():
    print(f"{k}: {v:.4f}")

Аналогичная процедура для трансформера

In [ ]:
gpt2 = transformer.DistilGPT2()
for i in range(0, NUM_WORDS):
    preds = gpt2.inference(gpt2_split)
    gpt2_split = [x + ' ' + y for x, y in zip(gpt2_split, preds)]

rouge_score = ROUGE.compute(predictions=gpt2_split, references=true_split)
print('ROUGE metrics')
for k, v in rouge_score.items():
    print(f"{k}: {v:.4f}")